In [24]:
!pip install --upgrade mlxtend

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules


In [4]:
df = pd.read_csv("dataset/bank.csv", sep=";")
df.head()




,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [5]:
df.shape

(4521, 17)

## Transform the dataset to "one hot table"

### Grouping the job from 12 diff types to 6 types

In [6]:
# Define the grouping mapping for the 'job' column (from 12 to 6)
job_group_mapping = {
    'management': 'white-collar',
    'technician': 'white-collar',
    'admin.': 'white-collar',
    'services': 'manual-labor',
    'blue-collar': 'manual-labor',
    'self-employed': 'entrepreneur',
    'entrepreneur': 'entrepreneur',
    'unemployed': 'no-job',
    'student': 'no-job',
    'retired': 'retired',
    'housemaid': 'manual-labor',
    'unknown': 'unknown'
}

df['job_grouped'] = df['job'].replace(job_group_mapping)
df['job'] = df['job_grouped']
df.drop('job_grouped', axis=1, inplace=True)



### Devide age'column into 4 parts: "adult", "middle-aged", "senior", "elderly"

In [7]:
# Define a mapping function for age groups
def map_age_to_group(age):
    if 15 <= age < 30:
        return 'adult'
    elif 30 <= age < 45:
        return 'middle-aged'
    elif 45 <= age < 65:
        return 'senior'
    else:
        return 'elderly'




In [8]:
# Apply the mapping function to the 'age' column
df['age_group'] = df['age'].apply(map_age_to_group)
df['age'] = df['age_group']
df.drop('age_group', axis=1, inplace=True)

# Display the first few rows of the updated dataset
df['age'].head()


0    middle-aged
1    middle-aged
2    middle-aged
3    middle-aged
4         senior
Name: age, dtype: object

### The Balance has the range between '-3313€' and '71188€' so i decide to devide it into 5 groups: Negative , Low, Medium , High , Very high .

In [9]:
def map_balance_to_group(balance):
    if balance < 0: 
        return  'negative'
    elif  0<= balance < 5000: 
        return 'low'
    elif 5000 <= balance < 20000:
        return 'medium'
    elif 20000 <= balance < 50000:
        return 'high'
    else:
        return 'very high'
    


In [10]:
df['balance_group'] = df['balance'].apply(map_balance_to_group) 
df['balance'] = df['balance_group']
df.drop('balance_group', axis=1, inplace=True)


In [11]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,middle-aged,no-job,married,primary,no,low,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,middle-aged,manual-labor,married,secondary,no,low,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,middle-aged,white-collar,single,tertiary,no,low,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,middle-aged,white-collar,married,tertiary,no,low,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,senior,manual-labor,married,secondary,no,low,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [12]:
df.rename(columns={'default': 'credit', 'y': 'subscribed_term_deposit'}, inplace=True)
df.head()

,age,job,marital,education,credit,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed_term_deposit
0,middle-aged,no-job,married,primary,no,low,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,middle-aged,manual-labor,married,secondary,no,low,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,middle-aged,white-collar,single,tertiary,no,low,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,middle-aged,white-collar,married,tertiary,no,low,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,senior,manual-labor,married,secondary,no,low,yes,no,unknown,5,may,226,1,-1,0,unknown,no


### We take the interests columns : 'age', 'job', 'marital', 'education', 'credit', 'balance', 'housing', 'loan', 'subscribed_term_deposit'

In [13]:
df.drop(['contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome'], axis=1, inplace=True)

In [14]:
df.head()

,age,job,marital,education,credit,balance,housing,loan,subscribed_term_deposit
0,middle-aged,no-job,married,primary,no,low,no,no,no
1,middle-aged,manual-labor,married,secondary,no,low,yes,yes,no
2,middle-aged,white-collar,single,tertiary,no,low,yes,no,no
3,middle-aged,white-collar,married,tertiary,no,low,yes,yes,no
4,senior,manual-labor,married,secondary,no,low,yes,no,no


### Transform to the "One hot table"

In [15]:
age_one_hot = pd.get_dummies(df['age'], prefix='age')
job_one_hot = pd.get_dummies(df['job'], prefix='job')
marital_one_hot = pd.get_dummies(df['marital'], prefix='marital')
education_one_hot = pd.get_dummies(df['education'], prefix='education')
credit_one_hot = pd.get_dummies(df['credit'], prefix='credit')
balance_one_hot = pd.get_dummies(df['balance'], prefix='balance')
housing_one_hot = pd.get_dummies(df['housing'], prefix='housing')
loan_one_hot = pd.get_dummies(df['loan'], prefix='loan')
subscribed_term_d_one_hot = pd.get_dummies(df['subscribed_term_deposit'], prefix='subscribed_term_d')

df = pd.concat([df, age_one_hot], axis=1)
df = pd.concat([df, job_one_hot], axis=1)
df = pd.concat([df, marital_one_hot], axis=1)
df = pd.concat([df, education_one_hot], axis=1)
df = pd.concat([df, credit_one_hot], axis=1)
df = pd.concat([df, balance_one_hot], axis=1)
df = pd.concat([df, housing_one_hot], axis=1)
df = pd.concat([df, loan_one_hot], axis=1)
df = pd.concat([df, subscribed_term_d_one_hot], axis=1)

df.drop(['age', 'job', 'marital', 'education', 'credit', 'balance', 'housing', 'loan', 'subscribed_term_deposit'], axis=1, inplace=True)

df.head(20)

,age_adult,age_elderly,age_middle-aged,age_senior,job_entrepreneur,job_manual-labor,job_no-job,job_retired,job_unknown,job_white-collar,...,balance_low,balance_medium,balance_negative,balance_very high,housing_no,housing_yes,loan_no,loan_yes,subscribed_term_d_no,subscribed_term_d_yes
0,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,1,0
1,0,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,1,1,0
2,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,1,1,0,1,0
3,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,1,0,1,1,0
4,0,0,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,1,0,1,0
5,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,0,1,0
6,0,0,1,0,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,1,0
7,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,1,1,0,1,0
8,0,0,1,0,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,1,0
9,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,1,1,0


In [16]:
df.shape

(4521, 30)

In [17]:
df.to_csv("dataset/bank_cleaned.csv", index=False)

## Experiment Frequent patterns and association rules discovery on data

### Compute frequent itemsets with interesting measures (at least support and tiles)

#### Threshold support = 0.25,  > 1131 occurancies

In [33]:
frequent_itemsets = apriori(df, min_support=0.25, use_colnames=True)
print(frequent_itemsets)

      support                                           itemsets
0    0.541031                                  (age_middle-aged)
1    0.332670                                       (age_senior)
2    0.326255                                 (job_manual-labor)
3    0.489936                                 (job_white-collar)
4    0.618668                                  (marital_married)
..        ...                                                ...
203  0.391064  (credit_no, balance_low, subscribed_term_d_no,...
204  0.270515  (credit_no, subscribed_term_d_no, marital_marr...
205  0.310993  (education_secondary, credit_no, balance_low, ...
206  0.265649  (credit_no, balance_low, subscribed_term_d_no,...
207  0.361867  (credit_no, balance_low, subscribed_term_d_no,...

[208 rows x 2 columns]


c:\Users\MAIYP\anaconda3\envs\crosswalk\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [62]:
frequent_itemsets['itemset_length'] = frequent_itemsets['itemsets'].apply(len)
frequent_itemsets['tiles'] = frequent_itemsets['support'] * frequent_itemsets['itemset_length']
frequent_itemsets.drop(['itemset_length'], axis=1, inplace=True)
frequent_itemsets

,support,itemsets,tiles
0,0.541031,(age_middle-aged),0.541031
1,0.332670,(age_senior),0.332670
2,0.326255,(job_manual-labor),0.326255
3,0.489936,(job_white-collar),0.489936
4,0.618668,(marital_married),0.618668
...,...,...,...
203,0.391064,"(credit_no, balance_low, subscribed_term_d_no,...",1.955320
204,0.270515,"(credit_no, subscribed_term_d_no, marital_marr...",1.352577
205,0.310993,"(education_secondary, credit_no, balance_low, ...",1.554966
206,0.265649,"(credit_no, balance_low, subscribed_term_d_no,...",1.328246


### Compute the "max_itemsets", "closed itemsets", "free itemsets", "generator itemsetes", "largets tiles"

In [92]:
#Compute the maximal itemsets
def is_maximal(itemset, all_itemsets):
    for other in all_itemsets:
        if set(itemset).issubset(set(other)) and set(itemset) != other:
            return False
    return True

frequent_itemsets['is_maximal'] = frequent_itemsets['itemsets'].apply(
    lambda x: is_maximal(x, frequent_itemsets['itemsets']))

maximal_itemsets = frequent_itemsets[frequent_itemsets['is_maximal'] == True]
print(maximal_itemsets)

      support                                           itemsets     tiles  \
22   0.251272                      (age_senior, marital_married)  0.502544   
44   0.259677                        (marital_single, credit_no)  0.519354   
69   0.250387   (age_middle-aged, balance_low, job_white-collar)  0.751161   
88   0.282238               (age_senior, balance_low, credit_no)  0.846715   
89   0.273391                   (age_senior, loan_no, credit_no)  0.820173   
90   0.288874      (age_senior, subscribed_term_d_no, credit_no)  0.866622   
91   0.250608    (age_senior, balance_low, subscribed_term_d_no)  0.751825   
93   0.268746             (job_manual-labor, loan_no, credit_no)  0.806238   
96   0.269852     (marital_married, job_white-collar, credit_no)  0.809555   
98   0.267861         (housing_yes, job_white-collar, credit_no)  0.803583   
109  0.256138           (housing_no, marital_married, credit_no)  0.768414   
129  0.252820       (balance_low, education_tertiary, credit_no)

In [93]:
# Compute Closed Itemsets
def is_closed(itemset, all_itemsets, all_supports):
    for other, support in zip(all_itemsets, all_supports):
        if set(itemset).issubset(other) and set(itemset) != other and support == all_supports[all_itemsets.index(other)]:
            return False
    return True

frequent_itemsets['is_closed'] = frequent_itemsets['itemsets'].apply(
    lambda x: is_closed(x, frequent_itemsets['itemsets'].tolist(), frequent_itemsets['support'].tolist())
)

closed_itemsets = frequent_itemsets[frequent_itemsets['is_closed'] == True]

print(closed_itemsets)

      support                                           itemsets     tiles  \
22   0.251272                      (age_senior, marital_married)  0.502544   
44   0.259677                        (marital_single, credit_no)  0.519354   
69   0.250387   (age_middle-aged, balance_low, job_white-collar)  0.751161   
88   0.282238               (age_senior, balance_low, credit_no)  0.846715   
89   0.273391                   (age_senior, loan_no, credit_no)  0.820173   
90   0.288874      (age_senior, subscribed_term_d_no, credit_no)  0.866622   
91   0.250608    (age_senior, balance_low, subscribed_term_d_no)  0.751825   
93   0.268746             (job_manual-labor, loan_no, credit_no)  0.806238   
96   0.269852     (marital_married, job_white-collar, credit_no)  0.809555   
98   0.267861         (housing_yes, job_white-collar, credit_no)  0.803583   
109  0.256138           (housing_no, marital_married, credit_no)  0.768414   
129  0.252820       (balance_low, education_tertiary, credit_no)

In [94]:
#Compute the generator itemsets
def is_generator(itemset, all_itemsets, all_supports):
    for other, support in zip(all_itemsets, all_supports):
        if set(itemset).issubset(other) and set(itemset) != other and support > all_supports[all_itemsets.index(other)]:
            return False
    return True

frequent_itemsets['is_generator'] = frequent_itemsets['itemsets'].apply(
    lambda x: is_generator(x, frequent_itemsets['itemsets'].tolist(), frequent_itemsets['support'].tolist())
)

generator_itemsets = frequent_itemsets[frequent_itemsets['is_generator'] == True]
print(generator_itemsets)

      support                                           itemsets     tiles  \
0    0.541031                                  (age_middle-aged)  0.541031   
1    0.332670                                       (age_senior)  0.332670   
2    0.326255                                 (job_manual-labor)  0.326255   
3    0.489936                                 (job_white-collar)  0.489936   
4    0.618668                                  (marital_married)  0.618668   
..        ...                                                ...       ...   
203  0.391064  (credit_no, balance_low, subscribed_term_d_no,...  1.955320   
204  0.270515  (credit_no, subscribed_term_d_no, marital_marr...  1.352577   
205  0.310993  (education_secondary, credit_no, balance_low, ...  1.554966   
206  0.265649  (credit_no, balance_low, subscribed_term_d_no,...  1.328246   
207  0.361867  (credit_no, balance_low, subscribed_term_d_no,...  1.809334   

     is_maximal  is_closed  is_generator  
0         False     

In [75]:
#Computer the largest tile
largest_tiles = frequent_itemsets['tiles'].max()
print(largest_tiles)

2.5100641451006416


In [77]:
frequent_itemsets.head(20)

,support,itemsets,tiles,is_maximal,is_closed,is_generator
0,0.541031,(age_middle-aged),0.541031,False,False,True
1,0.332670,(age_senior),0.332670,False,False,True
2,0.326255,(job_manual-labor),0.326255,False,False,True
3,0.489936,(job_white-collar),0.489936,False,False,True
4,0.618668,(marital_married),0.618668,False,False,True
5,0.264543,(marital_single),0.264543,False,False,True
6,0.510064,(education_secondary),0.510064,False,False,True
7,0.298607,(education_tertiary),0.298607,False,False,True
8,0.983190,(credit_no),0.983190,False,False,True
9,0.850697,(balance_low),0.850697,False,False,True


### Compute association rules  with interesting measures (lift, leverage, rule interest,  laplace, lovinger, ....)

In [96]:
# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6, num_itemsets=2)

print(rules)

                             antecedents  \
0                     (job_white-collar)   
1                      (age_middle-aged)   
2                      (age_middle-aged)   
3                      (age_middle-aged)   
4                          (housing_yes)   
..                                   ...   
737             (housing_yes, credit_no)   
738           (balance_low, housing_yes)   
739  (subscribed_term_d_no, housing_yes)   
740               (loan_no, housing_yes)   
741                        (housing_yes)   

                                           consequents  antecedent support  \
0                                    (age_middle-aged)            0.489936   
1                                          (credit_no)            0.541031   
2                                        (balance_low)            0.541031   
3                                        (housing_yes)            0.541031   
4                                    (age_middle-aged)            0.566025   
.. 

### Manage redundant rules

In [97]:
rules = rules.sort_values(by='lift', ascending=False).drop_duplicates(subset=['consequents'], keep='first')
print(rules)

                                           antecedents  \
7                                         (age_senior)   
349            (age_middle-aged, subscribed_term_d_no)   
346  (age_middle-aged, subscribed_term_d_no, credit...   
348     (subscribed_term_d_no, housing_yes, credit_no)   
323                         (balance_low, housing_yes)   
668        (balance_low, marital_married, housing_yes)   
667   (balance_low, subscribed_term_d_no, housing_yes)   
192                     (marital_married, housing_yes)   
666          (marital_married, housing_yes, credit_no)   
738                         (balance_low, housing_yes)   
670                     (marital_married, housing_yes)   
731              (balance_low, housing_yes, credit_no)   
536                            (housing_no, credit_no)   
230                               (education_tertiary)   
737                           (housing_yes, credit_no)   
229                    (education_tertiary, credit_no)   
533           